In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
filename = 'Mobility_trends.csv'

columns = ['country_region', 'sub_region_1', 'sub_region_2', 'date', 
           'retail_and_recreation_percent_change_from_baseline', 
           'grocery_and_pharmacy_percent_change_from_baseline',
           'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline',
           'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline']

df = pd.read_csv(filename, usecols = columns)

df = df.rename(columns={'country_region': 'country', 'sub_region_1': 'region',
                        'sub_region_2': 'subregion',
                        'retail_and_recreation_percent_change_from_baseline':'recreation',
                        'grocery_and_pharmacy_percent_change_from_baseline':'grocery and pharmacy',
                        'parks_percent_change_from_baseline': 'parks',
                        'transit_stations_percent_change_from_baseline':'transit',
                        'workplaces_percent_change_from_baseline': 'workplaces',
                        'residential_percent_change_from_baseline': 'residential'})

In [105]:
df.head()

,country,region,subregion,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
0,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


## Column descriptions

This dataset was collected on march 1st.

**Country**: represents which country the data is about

**sub_region_1** and **sub_region_2**: specify the region where this data takes place if available

**date**: when the data was collected, on august 17 2020 the data collection strategy was refined, so we will prioritize data onwards from there

The subsequent columns use a system of deviation from a baseline, which is calculated with the median value, for the corresponding day of the week, during the 5-week period Jan 3–Feb 6, 2020.

**recreation**: Mobility trends for places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.

**grocery and pharmacy**: Mobility trends for places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies.

**parks**: Mobility trends for places like local parks, national parks, public beaches, marinas, dog parks, plazas, and public gardens.

**transit**: Mobility trends for places like public transport hubs such as subway, bus, and train stations.

**workplaces**: Mobility trends for places of work.

**residential**: Mobility trends for places of residence.

## Empty values

In [106]:
print(f'The dataframe has {np.shape(df)[0]} rows')

The dataframe has 4389190 rows


In [107]:
100*df.isna().sum()/4389190

country                  0.000000
region                   1.700610
subregion               16.634003
date                     0.000000
recreation              37.200850
grocery and pharmacy    39.439145
parks                   51.937829
transit                 49.476464
workplaces               4.565808
residential             41.739410
dtype: float64

We can see that percentually there are a lot of missing values, probably because of uncooperative countries. But no missing dates. Ideally we would like to use data from after august 17th and try to eliminate countries with excessive missing values

## Dates

Dates are strings, so first we need to convert them to datetime and then eliminate unecessary values. 196 days passed since august 17th, therefore we expect somewhere around that for each time series.

In [108]:
df.loc[:,'date']=pd.to_datetime(df['date']).values
df = df.loc[df['date']>np.datetime64('2020-08-17').astype('datetime64[ns]')]
df.head()

,country,region,subregion,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
185,United Arab Emirates,NaN,NaN,2020-08-18,-23.0,-7.0,-42.0,-42.0,-21.0,11.0
186,United Arab Emirates,NaN,NaN,2020-08-19,-23.0,-7.0,-42.0,-42.0,-21.0,11.0
187,United Arab Emirates,NaN,NaN,2020-08-20,-26.0,-9.0,-45.0,-43.0,-22.0,12.0
188,United Arab Emirates,NaN,NaN,2020-08-21,-33.0,-16.0,-56.0,-50.0,-14.0,10.0
189,United Arab Emirates,NaN,NaN,2020-08-22,-23.0,-9.0,-43.0,-44.0,-15.0,8.0


In [109]:
print(f'The dataframe has {np.shape(df)[0]} rows')

The dataframe has 2221991 rows


In [110]:
100*df.isna().sum()/2221991

country                  0.000000
region                   1.706398
subregion               16.404747
date                     0.000000
recreation              39.670413
grocery and pharmacy    42.860885
parks                   53.106696
transit                 51.105202
workplaces               4.008477
residential             36.061172
dtype: float64

Percentage wise, the amount of missing data hasn't changed considerably. Which implies, as expected, that the missing values are due to countries/regions that don't collect properly.

## Exploring data by country and region

Our goal is to eliminate empty values. In order to do this we must first detect regions and subregions that arent counting properly and eliminate them. So the process will be the following:

Identify the countries, regions and sub regions. Starting from the sub regions we calculate the amount of NaN, if it is over a given % we discard that sub region. After eliminating all the empty sub-regions, we proceed to the regions that have NaN as their subregion, and then repeat for country.

### Sub-regions

In [629]:
subregions = df.subregion.unique()
subregions = subregions[1:] # Removes NaN
print(f'We have {len(subregions)} sub-regions')

We have 9899 sub-regions


In [630]:
#for i in subregions: print(i)

To see how much information our subregional data is missing:

In [631]:
(df.loc[~df.subregion.isna()].isna().sum()*100/len(df.loc[~df.subregion.isna()]))[4:]

recreation              44.026393
grocery and pharmacy    47.353752
parks                   58.437161
transit                 55.771182
workplaces               4.424438
residential             39.472909
dtype: float64

This shows that subregional data is slightly worse than region and nationwide data. Which makes sense. Thus, we need to clean our subregional data. We believe that most of the missing data is contained in specific subregions. Thus, if we exclude the problematic subregions our database will have decent enough data to impute linear estimates.

An example of the missing values in the sub region of Azul Partido

In [632]:
print((df[df.subregion == 'Azul Partido'].isna().sum())[4:])

recreation               22
grocery and pharmacy     46
parks                    22
transit                 108
workplaces                4
residential               2
dtype: int64


Considering we have 196 days, we expect around 190 days in our time series. Therefore, in order to keep a sub region, we want it to have less than 38 (20% of 190) missing values in each mobility trend category. Azul Partido, for example, shouldn't be kept. This code iterates by subregion and writes the ones without enough data on an array. It takes A VERY LONG TIME to run. We also decided to filter any subregions with less than 170 dates.

In [633]:
#unused_subregions = []
#for i in subregions:
    #if (df[df.subregion == i].isna().sum().nlargest(n=1)>38).any() or len(df[df.subregion == i]) < 170:
        #unused_subregions.append(i)

I ran it once so you don't have to and exported it to a .txt file. Just run the cells below to get it from the .txt please.

In [634]:
#with open('unused_subregions', 'w') as f:
    #for item in unused_subregions:
        #f.write("%s\n" % item)

In [635]:
my_file = open("unused_subregions", "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()
unused_subregions = content_list[:-1]

In [636]:
subregion_mask = df.subregion.isin(unused_subregions) # Creates a mask from the unused subregions
subregion_df = df.loc[~subregion_mask] # Removes every unused subregion from the dataset
subregion_df = subregion_df.loc[~subregion_df.subregion.isna()] # Removes every entry that doesnt have a subregion
subregion_df = subregion_df.reset_index()
subregion_df = subregion_df.drop(['index'], axis=1)

In [637]:
subregion_df.head()

,country,region,subregion,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
0,Argentina,Buenos Aires,Comuna 1,2020-08-18,-78.0,-32.0,-74.0,-73.0,-49.0,22.0
1,Argentina,Buenos Aires,Comuna 1,2020-08-19,-80.0,-33.0,-79.0,-75.0,-51.0,23.0
2,Argentina,Buenos Aires,Comuna 1,2020-08-20,-80.0,-36.0,-76.0,-75.0,-51.0,25.0
3,Argentina,Buenos Aires,Comuna 1,2020-08-21,-79.0,-28.0,-75.0,-73.0,-50.0,26.0
4,Argentina,Buenos Aires,Comuna 1,2020-08-22,-76.0,-9.0,-68.0,-70.0,-33.0,20.0


In [638]:
print(f'We have {len(subregion_df.subregion.unique())} subregions')

We have 2230 subregions


In [639]:
print(f'The subregion dataframe has {np.shape(subregion_df)[0]} rows')

The subregion dataframe has 423934 rows


In [640]:
(100*subregion_df.isna().sum()/423934)[4:]

recreation              3.238240
grocery and pharmacy    4.511551
parks                   7.540089
transit                 4.726443
workplaces              0.956045
residential             0.474838
dtype: float64

We can see that the subregion dataset has been cleared quite nicely. 

Linearly interpolating missing values reduces the NaN percentage, but it doesn't work when a subregion's first value onwards is NaN. I think we have three choices, we can either remove subregions with missing values altogether, remove the missing rows or just pad with the nearest value. I'm not sure which to do.

In [641]:
for i in subregion_df.subregion.unique():
    subregion_df.loc[subregion_df.subregion==i] = subregion_df.loc[subregion_df.subregion==i].interpolate()

In [642]:
(100*subregion_df.isna().sum()/423934)[4:]

recreation              2.368529
grocery and pharmacy    3.465870
parks                   6.360896
transit                 4.094741
workplaces              0.236593
residential             0.001179
dtype: float64

One solution is to cut all NaN blocks from above. So, to find where the largest NaN block ends:

In [643]:
max(subregion_df.loc[subregion_df.isnull().any(axis=1)].date)

Timestamp('2020-09-22 00:00:00')

In [644]:
subregion_df = subregion_df.loc[subregion_df.date>np.datetime64('2020-09-22').astype('datetime64[ns]')]

In [645]:
subregion_df.isna().sum()

country                 0
region                  0
subregion               0
date                    0
recreation              0
grocery and pharmacy    0
parks                   0
transit                 0
workplaces              0
residential             0
dtype: int64

With this our subregion dataset goes from 22.09.2020 to 23.02.2022, now we will ensure that all subregions have uninterrupted data.

In [646]:
aux = []
for i in subregion_df.subregion.unique():
    if len(subregion_df.loc[subregion_df.subregion == i]) != 154:
        aux.append(i)

In [647]:
aux

['Toledo', 'Cuenca', 'Aurangabad', 'El Paso County', 'Suffolk County']

In [648]:
subregion_mask = subregion_df.subregion.isin(aux) # Creates a mask from the unused subregions
subregion_df = subregion_df.loc[~subregion_mask] # Removes every unused subregion from the dataset
subregion_df = subregion_df.reset_index()
subregion_df = subregion_df.drop(['index'], axis=1)

In [649]:
print(f'We have {len(subregion_df.subregion.unique())} uninterrupted subregions')

We have 2225 uninterrupted subregions


In [650]:
subregion_df.to_csv(r'mobility trends datasets/subregion.csv', index = False)

### Regions

In [651]:
regions = df.region.unique()
regions = regions[1:]
print(f'We have {len(regions)} regions')

We have 1850 regions


In [652]:
#for i in regions: print(i)

Since we already have information on the subregions, we can ignore them and only observe completely regional data

In [653]:
region_df = df.loc[df.subregion.isna()] # Remove sub-regionwide entries
region_df = region_df.loc[~region_df.region.isna()] # Removes nationwide entries
region_df = region_df.drop(['subregion'], axis=1)
region_df.head()

,country,region,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
560,United Arab Emirates,Abu Dhabi,2020-08-18,-23.0,-11.0,-35.0,-54.0,-21.0,11.0
561,United Arab Emirates,Abu Dhabi,2020-08-19,-24.0,-11.0,-37.0,-55.0,-22.0,11.0
562,United Arab Emirates,Abu Dhabi,2020-08-20,-26.0,-13.0,-42.0,-58.0,-22.0,12.0
563,United Arab Emirates,Abu Dhabi,2020-08-21,-31.0,-19.0,-56.0,-67.0,-10.0,11.0
564,United Arab Emirates,Abu Dhabi,2020-08-22,-22.0,-13.0,-41.0,-60.0,-13.0,10.0


In [654]:
(region_df.isna().sum()*100/len(region_df))[3:]

recreation              19.251001
grocery and pharmacy    22.041605
parks                   28.602616
transit                 30.255423
workplaces               2.084226
residential             20.687639
dtype: float64

Unfortunately the regional dataframe needs to undergo the same cleaning the subregional one underwent.

In [655]:
unused_regions = []
for i in regions:
    if (region_df[region_df.region == i].isna().sum().nlargest(n=1)>38).any() or len(region_df[region_df.region == i]) < 170:
        unused_regions.append(i)

An example of a region with data collection issues.

In [656]:
print((region_df[region_df.region == 'Vraca'].isna().sum())[3:])

recreation              18
grocery and pharmacy    18
parks                   18
transit                 61
workplaces               0
residential             10
dtype: int64


In [657]:
region_mask = region_df.region.isin(unused_regions)
region_df = region_df.loc[~region_mask]
region_df = region_df.reset_index()
region_df = region_df.drop(['index'], axis=1)
region_df.head()

,country,region,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
0,United Arab Emirates,Abu Dhabi,2020-08-18,-23.0,-11.0,-35.0,-54.0,-21.0,11.0
1,United Arab Emirates,Abu Dhabi,2020-08-19,-24.0,-11.0,-37.0,-55.0,-22.0,11.0
2,United Arab Emirates,Abu Dhabi,2020-08-20,-26.0,-13.0,-42.0,-58.0,-22.0,12.0
3,United Arab Emirates,Abu Dhabi,2020-08-21,-31.0,-19.0,-56.0,-67.0,-10.0,11.0
4,United Arab Emirates,Abu Dhabi,2020-08-22,-22.0,-13.0,-41.0,-60.0,-13.0,10.0


In [658]:
print(f'We have {len(region_df.region.unique())} regions')

We have 1002 regions


With this the regions that collect data inefficiently were eliminated. We only have to linearly estimate individual NaN values.

In [659]:
(100*region_df.isna().sum()/len(region_df))[3:]

recreation              3.254676
grocery and pharmacy    3.476311
parks                   6.307149
transit                 4.958430
workplaces              0.905973
residential             0.409657
dtype: float64

Interpolating missing values.

In [660]:
for i in region_df.region.unique():
    region_df.loc[region_df.region==i] = region_df.loc[region_df.region==i].interpolate()

In [661]:
(100*region_df.isna().sum()/len(region_df))[3:]

recreation              2.612879
grocery and pharmacy    2.809304
parks                   5.309265
transit                 4.449510
workplaces              0.289386
residential             0.000000
dtype: float64

In [662]:
max(region_df.loc[region_df.isnull().any(axis=1)].date)

Timestamp('2020-09-12 00:00:00')

In [663]:
region_df = region_df.loc[region_df.date>np.datetime64('2020-09-12').astype('datetime64[ns]')]

In [664]:
region_df.isna().sum()

country                 0
region                  0
date                    0
recreation              0
grocery and pharmacy    0
parks                   0
transit                 0
workplaces              0
residential             0
dtype: int64

With this our region dataset goes from 12.09.2020 to 23.02.2022, totaling 164 days of uninterrupted complete data for 1002 different regions. 

In [665]:
aux = []
for i in region_df.region.unique():
    if len(region_df.loc[region_df.region == i]) != 164:
        aux.append(i)

In [666]:
aux

['Cordoba', 'Punjab']

In [667]:
region_mask = region_df.region.isin(aux)
region_df = region_df.loc[~region_mask]
region_df = region_df.reset_index()
region_df = region_df.drop(['index'], axis=1)

In [668]:
print(f'We have {len(region_df.region.unique())} regions')

We have 1000 regions


In [669]:
region_df.to_csv(r'mobility trends datasets/region.csv', index = False)

### Countries

In [670]:
countries = df.country.unique()
print(f'We have {len(countries)} countries')

We have 135 countries


In [671]:
#for i in countries: print(i)

In [672]:
country_df = df.loc[df.region.isna()]
country_df = country_df.drop(['region', 'subregion'], axis=1)
country_df.head()

,country,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
185,United Arab Emirates,2020-08-18,-23.0,-7.0,-42.0,-42.0,-21.0,11.0
186,United Arab Emirates,2020-08-19,-23.0,-7.0,-42.0,-42.0,-21.0,11.0
187,United Arab Emirates,2020-08-20,-26.0,-9.0,-45.0,-43.0,-22.0,12.0
188,United Arab Emirates,2020-08-21,-33.0,-16.0,-56.0,-50.0,-14.0,10.0
189,United Arab Emirates,2020-08-22,-23.0,-9.0,-43.0,-44.0,-15.0,8.0


In [673]:
(100*country_df.isna().sum()/len(country_df))[2:]

recreation              2.160038
grocery and pharmacy    2.088828
parks                   3.040933
transit                 2.115202
workplaces              0.205718
residential             1.345079
dtype: float64

Removing countries that don't collect data well

In [674]:
unused_countries = []
for i in countries:
    if (country_df[country_df.country == i].isna().sum().nlargest(n=1)>38).any():
        unused_countries.append(i)

In [675]:
country_mask = country_df.country.isin(unused_countries)
country_df = country_df.loc[~country_mask]
country_df = country_df.reset_index()
country_df = country_df.drop(['index'], axis=1)
country_df.head()

,country,date,recreation,grocery and pharmacy,parks,transit,workplaces,residential
0,United Arab Emirates,2020-08-18,-23.0,-7.0,-42.0,-42.0,-21.0,11.0
1,United Arab Emirates,2020-08-19,-23.0,-7.0,-42.0,-42.0,-21.0,11.0
2,United Arab Emirates,2020-08-20,-26.0,-9.0,-45.0,-43.0,-22.0,12.0
3,United Arab Emirates,2020-08-21,-33.0,-16.0,-56.0,-50.0,-14.0,10.0
4,United Arab Emirates,2020-08-22,-23.0,-9.0,-43.0,-44.0,-15.0,8.0


In [676]:
print(f'We have {len(country_df.country.unique())} different countries')

We have 125 different countries


In [677]:
(100*country_df.isna().sum()/len(country_df))[2:]

recreation              0.953911
grocery and pharmacy    0.748769
parks                   1.945432
transit                 1.514633
workplaces              0.071800
residential             0.041028
dtype: float64

Linearly interpolating missing values.

In [678]:
for i in countries:
    country_df.loc[country_df.country==i] = country_df.loc[country_df.country==i].interpolate()

In [679]:
(100*country_df.isna().sum()/len(country_df))[2:]

recreation              0.728255
grocery and pharmacy    0.557303
parks                   1.576176
transit                 1.278720
workplaces              0.000000
residential             0.000000
dtype: float64

Since the missing data starts from the first date and ends somewhere, we need to find the largest date with missing data. Which is:

In [680]:
max(country_df.loc[country_df.isnull().any(axis=1)].date)

Timestamp('2020-09-10 00:00:00')

In [681]:
country_df = country_df.loc[country_df.date>np.datetime64('2020-09-10').astype('datetime64[ns]')]

In [682]:
country_df.isna().sum()

country                 0
date                    0
recreation              0
grocery and pharmacy    0
parks                   0
transit                 0
workplaces              0
residential             0
dtype: int64

With this our country dataset goes from 09.09.2020 to 23.02.2022, totaling 167 days of uninterrupted complete data for 125 different countries. 

In [683]:
aux = []
for i in country_df.country.unique():
    if len(country_df.loc[country_df.country == i]) != 166:
        aux.append(i)

In [684]:
aux

['Egypt',
 'Georgia',
 'Ghana',
 'Haiti',
 'Jordan',
 'South Korea',
 'Kuwait',
 'Morocco',
 'Myanmar (Burma)',
 'Nicaragua',
 'Panama',
 'Philippines',
 'Qatar',
 'Serbia',
 'Tajikistan',
 'Ukraine',
 'Uruguay']

In [685]:
country_mask = country_df.country.isin(aux)
country_df = country_df.loc[~country_mask]
country_df = country_df.reset_index()
country_df = country_df.drop(['index'], axis=1)

In [686]:
print(f'We have {len(country_df.country.unique())} different countries')

We have 108 different countries


Many countries were lost because we are trying to get uninterrupted data, so having all 167 days is essential. However, in this step we noticed a few countries like Egypt with over 400 days worth of data, despite only analyzing 166. The person imputting is probably imputting more than one country's worth of data into Egypt. So, because of this we lost 17 countries. 

In [687]:
 x